# NYC Rolling Sales Data
Exploring and Analyzing Property Rolling Sales in New York City

- Link: https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page
- Glossary: https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf

### A. Importing and Cleaning Data
1. **Directories & Import**: Set directories, install package for importing excel, and import our data
2. **Extract Actual Data**: Remove titles rows, set the first row to be column names, and remove column name row from the data frame
3. **Variables & Data Types**: Import packages for data cleaning: dplyr, tidyr, and ggplot2. Rename variable names and convert variables to appropriate data types
4. **Transfer of Property**: Extract observations for transfer of property where sale price is zero
5. **Non-Transfer Properties**: Remove unreasonable observations to get the final clean_sales

#### A-1. Directories & Import

In [20]:
#Set Directories
setwd("D:/2018 Fall Baruch/CIS 3920 Data Mining")
getwd()

#Install readxl package
install.packages("readxl", repos = "http://cran.us.r-project.org")

[1] "D:/2018 Fall Baruch/CIS 3920 Data Mining"

Installing package into 'C:/Users/khanhdi/Documents/R/win-library/3.4'
(as 'lib' is unspecified)
Warning message:
"package 'readxl' is in use and will not be installed"

In [38]:
#Import package and data file
library(readxl)
rolling <- read_excel("rollingsales_manhattan.xls", col_names = FALSE)

In [39]:
sales <- rolling[-(1:4),]
colnames(sales) <- sales[1,]
sales <- sales[-1,]
head(sales, n = 5)

BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,NA,A4,189 EAST 7TH STREET,NA,...,1,0,1,987,2183,1860,1,A4,4844809,43242
1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,NA,A4,189 EAST 7TH STREET,NA,...,1,0,1,987,2183,1860,1,A4,0,43243
1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,390,35,NA,B1,113 AVENUE C,NA,...,2,0,2,1218,4764,1899,1,B1,0,43215
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,NA,C7,21-23 AVENUE B,NA,...,12,4,16,4186,10588,1900,2,C7,10350000,43350
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,NA,C1,200 EAST 7TH STREET,NA,...,18,0,18,2271,10650,1910,2,C1,5000000,43348


In [22]:
library(dplyr)
library(tidyr)
library(ggplot2)

In [23]:
str(sales)

Classes 'tbl_df', 'tbl' and 'data.frame':	16750 obs. of  21 variables:
 $ BOROUGH                       : chr  "1" "1" "1" "1" ...
 $ NEIGHBORHOOD                  : chr  "ALPHABET CITY" "ALPHABET CITY" "ALPHABET CITY" "ALPHABET CITY" ...
 $ BUILDING CLASS CATEGORY       : chr  "01 ONE FAMILY DWELLINGS" "01 ONE FAMILY DWELLINGS" "02 TWO FAMILY DWELLINGS" "07 RENTALS - WALKUP APARTMENTS" ...
 $ TAX CLASS AT PRESENT          : chr  "1" "1" "1" "2" ...
 $ BLOCK                         : chr  "390" "390" "390" "385" ...
 $ LOT                           : chr  "61" "61" "35" "2" ...
 $ EASE-MENT                     : chr  NA NA NA NA ...
 $ BUILDING CLASS AT PRESENT     : chr  "A4" "A4" "B1" "C7" ...
 $ ADDRESS                       : chr  "189 EAST 7TH STREET" "189 EAST 7TH STREET" "113 AVENUE C" "21-23 AVENUE B" ...
 $ APARTMENT NUMBER              : chr  NA NA NA NA ...
 $ ZIP CODE                      : chr  "10009" "10009" "10009" "10009" ...
 $ RESIDENTIAL UNITS             : chr  "1"

In [24]:
names(sales) <- c('Borough', 'Neighborhood', 'BuildingClassCategory', 'TaxClassAtPresent', 'Block', 'Lot', 'EaseMent', 
                 'BuildingClassAtPresent', 'Address', 'ApartmentNumber', 'ZipCode', 'ResidentialUnits', 
                 'CommercialUnits', 'TotalUnits', 'LandSquareFeet', 'GrossSquareFeet', 
                 'YearBuilt', 'TaxClassAtTimeOfSale', 'BuildingClassAtTimeOfSale', 'SalePrice', 'SaleDate')
names(sales)

[1] "Borough"                   "Neighborhood"             
 [3] "BuildingClassCategory"     "TaxClassAtPresent"        
 [5] "Block"                     "Lot"                      
 [7] "EaseMent"                  "BuildingClassAtPresent"   
 [9] "Address"                   "ApartmentNumber"          
[11] "ZipCode"                   "ResidentialUnits"         
[13] "CommercialUnits"           "TotalUnits"               
[15] "LandSquareFeet"            "GrossSquareFeet"          
[17] "YearBuilt"                 "TaxClassAtTimeOfSale"     
[19] "BuildingClassAtTimeOfSale" "SalePrice"                
[21] "SaleDate"

In [25]:
factors <- c("Borough", "Neighborhood", "BuildingClassCategory", "TaxClassAtPresent", "BuildingClassAtPresent", 
             "TaxClassAtTimeOfSale", "BuildingClassAtTimeOfSale")
numerics <- c("ZipCode", "ResidentialUnits", "CommercialUnits", "YearBuilt", 
              "TotalUnits", "LandSquareFeet", "GrossSquareFeet", "SalePrice")

sales <- sales %>% mutate_at(factors, as.factor) 
sales <- sales %>% mutate_at(numerics, as.numeric)

str(sales)

Classes 'tbl_df', 'tbl' and 'data.frame':	16750 obs. of  21 variables:
 $ Borough                  : Factor w/ 1 level "1": 1 1 1 1 1 1 1 1 1 1 ...
 $ Neighborhood             : Factor w/ 39 levels "ALPHABET CITY",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ BuildingClassCategory    : Factor w/ 43 levels "01 ONE FAMILY DWELLINGS",..: 1 1 2 6 6 6 6 6 6 6 ...
 $ TaxClassAtPresent        : Factor w/ 9 levels "1","1A","1C",..: 1 1 1 4 4 5 6 4 4 4 ...
 $ Block                    : chr  "390" "390" "390" "385" ...
 $ Lot                      : chr  "61" "61" "35" "2" ...
 $ EaseMent                 : chr  NA NA NA NA ...
 $ BuildingClassAtPresent   : Factor w/ 129 levels "A1","A4","A5",..: 2 2 6 17 11 13 14 14 14 11 ...
 $ Address                  : chr  "189 EAST 7TH STREET" "189 EAST 7TH STREET" "113 AVENUE C" "21-23 AVENUE B" ...
 $ ApartmentNumber          : chr  NA NA NA NA ...
 $ ZipCode                  : num  10009 10009 10009 10009 10009 ...
 $ ResidentialUnits         : num  1 1 2 12 18 4 8 17 4

In [26]:
summary(sales[,-1])

                  Neighborhood                        BuildingClassCategory
 UPPER EAST SIDE (59-79):1892   13 CONDOS - ELEVATOR APARTMENTS  :6704     
 UPPER EAST SIDE (79-96):1536   10 COOPS - ELEVATOR APARTMENTS   :5674     
 UPPER WEST SIDE (59-79):1527   17 CONDO COOPS                   : 815     
 MIDTOWN EAST           :1162   09 COOPS - WALKUP APARTMENTS     : 697     
 MIDTOWN WEST           : 833   07 RENTALS - WALKUP APARTMENTS   : 627     
 UPPER WEST SIDE (79-96): 811   15 CONDOS - 2-10 UNIT RESIDENTIAL: 271     
 (Other)                :8989   (Other)                          :1962     
 TaxClassAtPresent    Block               Lot              EaseMent        
 2      :14176     Length:16750       Length:16750       Length:16750      
 4      : 1209     Class :character   Class :character   Class :character  
 2C     :  573     Mode  :character   Mode  :character   Mode  :character  
 1      :  345                                                             
 2B     :  2

#### Transfer of Property

Sales Prices that has zero values are transferred property. They are likely to be gifts, inheritance from parents to their children, or transfer of properties between family members so there are no money involved. We will filter these observations and assign them to 'transfer'

In [36]:
transfer <- sales %>% filter(SalePrice == 0 & TotalUnits != 0 & ZipCode != 0 &
                             YearBuilt != 0 & GrossSquareFeet != 0) #pipe operator
head(transfer, n = 5)
str(transfer)

Borough,Neighborhood,BuildingClassCategory,TaxClassAtPresent,Block,Lot,EaseMent,BuildingClassAtPresent,Address,ApartmentNumber,...,ResidentialUnits,CommercialUnits,TotalUnits,LandSquareFeet,GrossSquareFeet,YearBuilt,TaxClassAtTimeOfSale,BuildingClassAtTimeOfSale,SalePrice,SaleDate
1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,NA,A4,189 EAST 7TH STREET,NA,...,1,0,1,987,2183,1860,1,A4,0,43243
1,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,390,35,NA,B1,113 AVENUE C,NA,...,2,0,2,1218,4764,1899,1,B1,0,43215
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,392,22,NA,C4,374 EAST 10TH STREET,NA,...,17,0,17,4614,20599,1900,2,C4,0,43279
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,392,27,NA,C4,384 EAST 10TH STREET,NA,...,45,0,45,9234,49799,1900,2,C4,0,43279
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,392,48,NA,C1,627 EAST 9TH STREET,NA,...,35,0,35,9227,43871,1900,2,C1,0,43279


Classes 'tbl_df', 'tbl' and 'data.frame':	782 obs. of  21 variables:
 $ Borough                  : Factor w/ 1 level "1": 1 1 1 1 1 1 1 1 1 1 ...
 $ Neighborhood             : Factor w/ 39 levels "ALPHABET CITY",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ BuildingClassCategory    : Factor w/ 43 levels "01 ONE FAMILY DWELLINGS",..: 1 2 6 6 6 6 6 6 6 6 ...
 $ TaxClassAtPresent        : Factor w/ 9 levels "1","1A","1C",..: 1 1 4 4 4 4 4 4 4 4 ...
 $ Block                    : chr  "390" "390" "392" "392" ...
 $ Lot                      : chr  "61" "35" "22" "27" ...
 $ EaseMent                 : chr  NA NA NA NA ...
 $ BuildingClassAtPresent   : Factor w/ 129 levels "A1","A4","A5",..: 2 6 14 14 11 17 17 17 17 17 ...
 $ Address                  : chr  "189 EAST 7TH STREET" "113 AVENUE C" "374 EAST 10TH STREET" "384 EAST 10TH STREET" ...
 $ ApartmentNumber          : chr  NA NA NA NA ...
 $ ZipCode                  : num  10009 10009 10009 10009 10009 ...
 $ ResidentialUnits         : num  1 2 17 45 35 

#### Non-Transfer Sales

In [32]:
clean_sales <- sales %>% filter(SalePrice != 0 & TotalUnits != 0 & ZipCode != 0 &
                                YearBuilt != 0 & GrossSquareFeet != 0)
head(clean_sales, n = 5)
str(clean_sales)

Borough,Neighborhood,BuildingClassCategory,TaxClassAtPresent,Block,Lot,EaseMent,BuildingClassAtPresent,Address,ApartmentNumber,...,ResidentialUnits,CommercialUnits,TotalUnits,LandSquareFeet,GrossSquareFeet,YearBuilt,TaxClassAtTimeOfSale,BuildingClassAtTimeOfSale,SalePrice,SaleDate
1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,390,61,NA,A4,189 EAST 7TH STREET,NA,...,1,0,1,987,2183,1860,1,A4,4844809,43242
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,2,NA,C7,21-23 AVENUE B,NA,...,12,4,16,4186,10588,1900,2,C7,10350000,43350
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,389,18,NA,C1,200 EAST 7TH STREET,NA,...,18,0,18,2271,10650,1910,2,C1,5000000,43348
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,390,54,NA,C3,203 EAST 7TH STREET,NA,...,4,0,4,1950,5446,2001,2,C3,6250000,43229
1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,390,64,NA,C4,187 EAST 7TH STREET,NA,...,8,2,10,1642,5220,1910,2,C4,2400000,43238


Classes 'tbl_df', 'tbl' and 'data.frame':	837 obs. of  21 variables:
 $ Borough                  : Factor w/ 1 level "1": 1 1 1 1 1 1 1 1 1 1 ...
 $ Neighborhood             : Factor w/ 39 levels "ALPHABET CITY",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ BuildingClassCategory    : Factor w/ 43 levels "01 ONE FAMILY DWELLINGS",..: 1 6 6 6 6 6 6 6 6 6 ...
 $ TaxClassAtPresent        : Factor w/ 9 levels "1","1A","1C",..: 1 4 4 5 6 4 4 4 4 4 ...
 $ Block                    : chr  "390" "385" "389" "390" ...
 $ Lot                      : chr  "61" "2" "18" "54" ...
 $ EaseMent                 : chr  NA NA NA NA ...
 $ BuildingClassAtPresent   : Factor w/ 129 levels "A1","A4","A5",..: 2 17 11 13 14 14 17 17 17 17 ...
 $ Address                  : chr  "189 EAST 7TH STREET" "21-23 AVENUE B" "200 EAST 7TH STREET" "203 EAST 7TH STREET" ...
 $ ApartmentNumber          : chr  NA NA NA NA ...
 $ ZipCode                  : num  10009 10009 10009 10009 10009 ...
 $ ResidentialUnits         : num  1 12 18 4 8 1

In [33]:
summary(clean_sales[,-1])

                  Neighborhood                      BuildingClassCategory
 HARLEM-CENTRAL         :131   07 RENTALS - WALKUP APARTMENTS  :290      
 UPPER EAST SIDE (79-96): 63   01 ONE FAMILY DWELLINGS         : 99      
 UPPER EAST SIDE (59-79): 58   08 RENTALS - ELEVATOR APARTMENTS: 82      
 GREENWICH VILLAGE-WEST : 45   02 TWO FAMILY DWELLINGS         : 63      
 UPPER WEST SIDE (79-96): 41   21 OFFICE BUILDINGS             : 56      
 HARLEM-UPPER           : 38   22 STORE BUILDINGS              : 51      
 (Other)                :461   (Other)                         :196      
 TaxClassAtPresent    Block               Lot              EaseMent        
 2      :259       Length:837         Length:837         Length:837        
 1      :198       Class :character   Class :character   Class :character  
 4      :192       Mode  :character   Mode  :character   Mode  :character  
 2B     :107                                                               
 2A     : 76                